# Imports

In [ ]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

# Parameters

In [ ]:
sbw_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG\230928 Richterswil SK\Alle_exp_SBW\Exports\Teileinzugsgebiete_SBW.shp")
quartiere_path = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Bevölkerungsanalyse\Quartieranalyse\Daten\Quartieranalyse\QUARTIERE_F.shp")
sbw_tabelle = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil"
    r"\05 Berechnungen Grundlagen\8 Entwässerungskonzept\SBW im EZG\Richterswil_SBW-Tabelle.csv")

# Analysis

## EW pro Sonderbauwerk

In [ ]:
sbw = gpd.read_file(sbw_path)
quartiere = gpd.read_file(quartiere_path)
df_sbw = pd.read_csv(sbw_tabelle, sep=';')

In [ ]:
# EW dichte GEP
sbw['ew_dichte_gep'] = sbw['einwohner'] / sbw.area
print(sbw['einwohner'].sum())

# EW dichte Quartieranalyse
quartiere['ew_dichte_quartiere'] = (quartiere['U_EINW'] + quartiere['U_BESCH']) / quartiere.area
print(quartiere['U_EINW'].sum() + quartiere['U_BESCH'].sum())

In [ ]:
print(sbw.geometry.type.unique())
print(quartiere.geometry.type.unique())

sbw_quartiere = sbw.overlay(quartiere, how='intersection', keep_geom_type=None)
sbw_quartiere.rename(columns={'sbw_ref': 'SK ID'}, inplace=True)
print(sbw_quartiere.geometry.type.unique())

In [ ]:
sbw_quartiere['einwohner_gep'] = sbw_quartiere['ew_dichte_gep'] * sbw_quartiere.area
print(sbw_quartiere['einwohner_gep'].sum())

sbw_quartiere['einwohner_quartier'] = sbw_quartiere['ew_dichte_quartiere'] * sbw_quartiere.area
print(sbw_quartiere['einwohner_quartier'].sum())

In [ ]:
ew_gep = sbw_quartiere.groupby('SK ID')['einwohner_gep'].sum()
ew_gep = pd.merge(ew_gep, df_sbw, on='SK ID')
ew_gep = ew_gep[['Name', 'einwohner_gep']]
ew_gep.set_index(inplace=True, keys='Name')
ew_gep.sort_index(inplace=True)
ew_gep

In [ ]:
ew_quartiere = sbw_quartiere.groupby('SK ID')['einwohner_quartier'].sum()
ew_quartiere = pd.merge(ew_quartiere, df_sbw, on='SK ID')
ew_quartiere = ew_quartiere[['Name', 'einwohner_quartier']]
ew_quartiere.set_index(inplace=True, keys='Name')
ew_quartiere.sort_index(inplace=True)
print(ew_quartiere.sum())
ew_quartiere

In [ ]:
ew_sbw = pd.merge(ew_gep, ew_quartiere, on='Name')
ew_sbw.loc['Summe','einwohner_gep'] = ew_sbw.iloc[:,0].sum()
ew_sbw.loc['Summe','einwohner_quartier'] = ew_sbw.iloc[:,1].sum()

In [ ]:
ew_sbw

## EW pro Bauzone